In [1]:
import pandas as pd
import progressbar
pbar = progressbar.ProgressBar()
pd.options.mode.chained_assignment = None

In [2]:
pf = pd.read_json("Week 14/portfolio.json", lines=True)
pf.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [3]:
pf = pd.read_json("Week 14/profile.json", lines=True)
pf.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [ ]:
pf = pd.read_json("Week 14/transcript.json", lines=True)

for indx in pbar(pf["value"].index):
    try:
        pf["value"][indx] = pf["value"][indx]["offer id"]
    except:
        continue

pf.head()

<ipython-input-5-a8e0afbe52fe>:5: SettingWithCopyWarning:                     |
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pf["value"][indx] = pf["value"][indx]["offer id"]


In [7]:
pf.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0
2,e2127556f4f64592b11af22de27a7932,offer received,2906b810c7d4411798c6938adc9daaa5,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,fafdcd668e3743c1bb461111dcafc2a4,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,0


In [2]:
import base64
import os
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display_html
from matplotlib.figure import Figure
from sklearn.preprocessing import robust_scale, quantile_transform, scale

In [3]:
## Global definitions

data_dir = 'Week 14'

pd.set_option('display.precision', 2)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

portfolio_data_path = os.path.join(data_dir, 'portfolio.json')
profile_data_path = os.path.join(data_dir, 'profile.json')
transcript_data_path = os.path.join(data_dir, 'transcript.json')

In [4]:
## global functions

def load_dataframe(data_path):
    """Create a dataframe from a json file"""
    return pd.read_json(data_path, orient='records', lines=True)


def convert_fig_to_html(figure):
    """
    Convert a given matplotlib figure to HTML code
    
    Attributes:
    figure (matplotlib.figure.Figure): Generated without using pyplot.
    
    Return:
    return: img-html tag containing the figure data
    """
    # Save figure to a temporary buffer
    buf = BytesIO()
    figure.savefig(buf, format="png")
    # Embed the result in the html output
    data = base64.b64encode(buf.getbuffer()).decode("ascii")
    return f'<img src="data:image/png;base64,{data}" ' \
            'style="display:inline;top:0%"/>'

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

portfolio_df = load_dataframe(portfolio_data_path)

X = vect.fit_transform(portfolio_df.channels.str.join(' '))
portfolio_df = portfolio_df.join(pd.DataFrame(X.toarray(), columns=vect.get_feature_names()))

portfolio_df = portfolio_df.join(pd.get_dummies(portfolio_df["offer_type"]))
portfolio_df = portfolio_df.drop(columns=['channels', 'offer_type'])

portfolio_df.duration = portfolio_df.duration * 24

portfolio_df

,reward,difficulty,duration,id,email,mobile,social,web,bogo,discount,informational
0,10,10,168,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0,0
1,10,10,120,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0
2,0,0,96,3f207df678b143eea3cee63160fa8bed,1,1,0,1,0,0,1
3,5,5,168,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,1,0,0
4,5,20,240,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,0,1,0
5,3,7,168,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,0,1,0
6,2,10,240,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,0,1,0
7,0,0,72,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0,0,0,1
8,5,5,120,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,1,0,0
9,2,10,168,2906b810c7d4411798c6938adc9daaa5,1,1,0,1,0,1,0


In [34]:
profile_df = load_dataframe(profile_data_path)
profile_df.age = profile_df.age.replace(118, np.nan)
profile_df.dropna(subset=['age',])

,gender,age,id,became_member_on,income
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
5,M,68.0,e2127556f4f64592b11af22de27a7932,20180426,70000.0
8,M,65.0,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
12,M,58.0,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0
...,...,...,...,...,...
16995,F,45.0,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61.0,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49.0,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83.0,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [59]:
def clean_transcript(transcript):
    '''
    data cleaning the transcript dataframe, return usable dataframe need contain offer events
    
    INPUT:
    transcript - the transcript dataframe to be cleaned
    
    OUTPUT:
    offer - the cleaned offer dataframe
    
    '''
    # extract the rows that are related to offer action (e.g. offer received, offer viewed, offer completed)
    offer = transcript[transcript['value'].apply(lambda x: True if ('offer id' in x) or ('offer_id' in x) else False)]
    
    # extract the offer id from value column
    offer['offer_id'] = offer['value'].apply(lambda x: x['offer id'] if ('offer id' in x) else x['offer_id'])
    
    return offer

def get_transactions(transcript):
    txn = transcript[transcript['value'].apply(lambda x: True if ('amount' in x) else False)]
    txn['amount'] = txn['value'].apply(lambda x: x['amount'])
    return txn

def get_rewards(transcript):
    reward = transcript[transcript['value'].apply(lambda x: True if ('offer_id' in x) else False)]
    
    reward['offer_id'] = reward['value'].apply(lambda x: x['offer_id'])
    reward['reward'] = reward['value'].apply(lambda x: x['reward'])
    return reward

In [60]:
reward_df = get_rewards(transcript_df)
display(reward_df)

,person,event,value,time,offer_id,reward
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}",0,2906b810c7d4411798c6938adc9daaa5,2
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,"{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2}",0,fafdcd668e3743c1bb461111dcafc2a4,2
12679,629fc02d56414d91bca360decdfa9288,offer completed,"{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9', 'reward': 5}",0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5
12692,676506bad68e4161b9bbaffeb039626b,offer completed,"{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd', 'reward': 10}",0,ae264e3637204a6fb9bb56bc8210ddfd,10
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,"{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0', 'reward': 10}",0,4d5c57ea9a6940dd891ad53e9dbe8da0,10
...,...,...,...,...,...,...
306475,0c027f5f34dd4b9eba0a25785c611273,offer completed,"{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2', 'reward': 3}",714,2298d6c36e964ae4a3e7e9706d1fb8c2,3
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,"{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2', 'reward': 3}",714,2298d6c36e964ae4a3e7e9706d1fb8c2,3
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,"{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2}",714,fafdcd668e3743c1bb461111dcafc2a4,2
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,"{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2}",714,fafdcd668e3743c1bb461111dcafc2a4,2


In [52]:
transcript_df = load_dataframe(transcript_data_path)
offer_df = clean_transcript(transcript_df)
display(offer_df)

txn_df = get_transactions(transcript_df)
display(txn_df)

,person,event,value,time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,"{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2', 'reward': 3}",714,2298d6c36e964ae4a3e7e9706d1fb8c2
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,"{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2}",714,fafdcd668e3743c1bb461111dcafc2a4
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},714,0b1e1539f2cc45b7b9fa7c272da2e1d7
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,"{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2}",714,fafdcd668e3743c1bb461111dcafc2a4


,person,event,value,time,amount
12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,{'amount': 34.56},0,34.56
12659,54890f68699049c2a04d415abc25e717,transaction,{'amount': 13.23},0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,{'amount': 19.51},0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,{'amount': 18.97},0,18.97
...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,3.53


In [53]:
portfolio_df.to_excel("portfolio_df.xlsx")
profile_df.to_excel("profile_df.xlsx")
offer_df.to_excel("offer_df.xlsx")
txn_df.to_excel("txn_df.xlsx")

In [61]:
reward_df.to_excel("cleaned/reward_df.xlsx")

In [69]:
import pandas as pd
import numpy as np
profile_df = pd.read_excel("cleaned/profile_df.xlsx")
portfolio_df = pd.read_excel("cleaned/portfolio_df.xlsx") # this has offer_id
offer_df = pd.read_excel("cleaned/offer_df.xlsx")
txn_df = pd.read_excel("cleaned/txn_df.xlsx")
reward_df = pd.read_excel("cleaned/reward_df.xlsx")

profile_df["count"]=1

In [117]:
new = pd.merge(profile_df,offer_df,on='customer_id').groupby(["customer_id", "event"], as_index=False)
new = new["count"].agg("sum")

new2 = pd.DataFrame(columns=["customer_id", "offer completed", "offer received", "offer viewed"])
for indx in new.index:
    if "completed" in new["event"][indx]:
        dict1 = {
            "customer_id": new["customer_id"][indx],
            "offer completed": int(new["count"][indx]),
            "offer received": 0,
            "offer viewed": 0
                }
        new2 = new2.append(dict1, ignore_index=True)

#         new2["customer_id"][indx] = new["customer_id"][indx]
#         new2["offer completed"][indx] = int(new["count"][indx])
#         new2["offer received"][indx] = 0
#         new2["offer viewed"][indx] = 0
    elif "received" in new["event"][indx]:
        dict1 = {
            "customer_id": new["customer_id"][indx],
            "offer completed": 0,
            "offer received": int(new["count"][indx]),
            "offer viewed": 0
                }
        new2 = new2.append(dict1, ignore_index=True)
        
#         new2["customer_id"][indx] = new["customer_id"][indx]
#         new2["offer completed"][indx] = 0
#         new2["offer received"][indx] = int(new["count"][indx])
#         new2["offer viewed"][indx] = 0
    elif "viewed" in new["event"][indx]:
        dict1 = {
            "customer_id": new["customer_id"][indx],
            "offer completed": 0,
            "offer received": 0,
            "offer viewed": int(new["count"][indx])
                }
        new2 = new2.append(dict1, ignore_index=True)
        
#         new2["customer_id"][indx] = new["customer_id"][indx]
#         new2["offer completed"][indx] = 0
#         new2["offer received"][indx] = 0
#         new2["offer viewed"][indx] = int(new["count"][indx])
    else:
        dict1 = {
            "customer_id": new["customer_id"][indx],
            "offer completed": 0,
            "offer received": 0,
            "offer viewed": 0
                }
        new2 = new2.append(dict1, ignore_index=True)
#         continue
    

new2

,customer_id,offer completed,offer received,offer viewed
0,0009655768c64bdeb2e877511632db8f,3,0,0
1,0009655768c64bdeb2e877511632db8f,0,5,0
2,0009655768c64bdeb2e877511632db8f,0,0,4
3,00116118485d4dfda04fdbaba9a87b5c,0,2,0
4,00116118485d4dfda04fdbaba9a87b5c,0,0,2
...,...,...,...,...
46597,fffad4f4828548d1b5583907f2e9906b,0,4,0
46598,fffad4f4828548d1b5583907f2e9906b,0,0,4
46599,ffff82501cea40309d5fdd7edcca4a07,6,0,0
46600,ffff82501cea40309d5fdd7edcca4a07,0,6,0


In [118]:
new2.head(10)

,customer_id,offer completed,offer received,offer viewed
0,0009655768c64bdeb2e877511632db8f,3,0,0
1,0009655768c64bdeb2e877511632db8f,0,5,0
2,0009655768c64bdeb2e877511632db8f,0,0,4
3,00116118485d4dfda04fdbaba9a87b5c,0,2,0
4,00116118485d4dfda04fdbaba9a87b5c,0,0,2
5,0011e0d4e6b944f998e987f904e8c1e5,3,0,0
6,0011e0d4e6b944f998e987f904e8c1e5,0,5,0
7,0011e0d4e6b944f998e987f904e8c1e5,0,0,5
8,0020c2b971eb4e9188eac86d93036a77,3,0,0
9,0020c2b971eb4e9188eac86d93036a77,0,5,0


In [125]:
new2.to_excel("new2.xlsx")

In [130]:
aggregation_func = {
    "offer completed": "sum",
    "offer received": "sum",
    "offer viewed": "sum",
}

df_new = new2.groupby(["customer_id"], as_index=False).agg(aggregation_func)
df_new

,customer_id,offer completed,offer received,offer viewed
0,0009655768c64bdeb2e877511632db8f,3,5,4
1,00116118485d4dfda04fdbaba9a87b5c,0,2,2
2,0011e0d4e6b944f998e987f904e8c1e5,3,5,5
3,0020c2b971eb4e9188eac86d93036a77,3,5,3
4,0020ccbbb6d84e358d3414a3ff76cffd,3,4,4
...,...,...,...,...
16989,fff3ba4757bd42088c044ca26d73817a,3,6,3
16990,fff7576017104bcc8677a8d63322b5e1,3,5,4
16991,fff8957ea8b240a6b5e634b6ee8eafcf,0,3,2
16992,fffad4f4828548d1b5583907f2e9906b,3,4,4


In [132]:
new = pd.merge(profile_df,df_new,on='customer_id')
# new = pd.merge(offer_count,new,on='customer_id', how='right')
combined_transposed = new
combined_transposed

,gender,age,customer_id,became_member_on,income,count,offer completed,offer received,offer viewed
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,1,2,5,5
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,1,2,0
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN,1,0,2,2
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,3,4,4
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,1,0,5,3
...,...,...,...,...,...,...,...,...,...
16989,F,45.0,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0,1,0,3,3
16990,M,61.0,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0,1,1,3,1
16991,M,49.0,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0,1,0,3,1
16992,F,83.0,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0,1,3,3,3


In [133]:
customer_offer_combined = pd.merge(combined_transposed,offer_df,on='customer_id')
customer_offer_combined

,gender,age,customer_id,became_member_on,income,count,offer completed,offer received,offer viewed,event,value,time,offer_id
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,1,2,5,5,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},168,2906b810c7d4411798c6938adc9daaa5
1,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,1,2,5,5,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},216,2906b810c7d4411798c6938adc9daaa5
2,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,1,2,5,5,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},336,0b1e1539f2cc45b7b9fa7c272da2e1d7
3,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,1,2,5,5,offer viewed,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},348,0b1e1539f2cc45b7b9fa7c272da2e1d7
4,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,1,2,5,5,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},408,fafdcd668e3743c1bb461111dcafc2a4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167576,F,62.0,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,1,2,4,2,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed
167577,F,62.0,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,1,2,4,2,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d
167578,F,62.0,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,1,2,4,2,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,480,f19421c1d4aa40978ebb69ca19b0e20d
167579,F,62.0,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,1,2,4,2,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},546,f19421c1d4aa40978ebb69ca19b0e20d


In [135]:
customer_offer_portfolio_combined = pd.merge(customer_offer_combined,portfolio_df,on='offer_id')
customer_offer_portfolio_combined

,gender,age,customer_id,became_member_on,income,count,offer completed,offer received,offer viewed,event,...,reward,difficulty,duration,email,mobile,social,web,bogo,discount,informational
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,1,2,5,5,offer received,...,2,10,168,1,1,0,1,0,1,0
1,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,1,2,5,5,offer viewed,...,2,10,168,1,1,0,1,0,1,0
2,M,68.0,e2127556f4f64592b11af22de27a7932,20180426,70000.0,1,2,4,3,offer received,...,2,10,168,1,1,0,1,0,1,0
3,M,68.0,e2127556f4f64592b11af22de27a7932,20180426,70000.0,1,2,4,3,offer viewed,...,2,10,168,1,1,0,1,0,1,0
4,NaN,NaN,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN,1,0,5,5,offer received,...,2,10,168,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167576,F,45.0,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0,1,0,3,3,offer received,...,10,10,120,1,1,1,1,1,0,0
167577,F,45.0,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0,1,0,3,3,offer viewed,...,10,10,120,1,1,1,1,1,0,0
167578,F,83.0,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0,1,3,3,3,offer received,...,10,10,120,1,1,1,1,1,0,0
167579,F,83.0,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0,1,3,3,3,offer viewed,...,10,10,120,1,1,1,1,1,0,0


In [136]:
combined_transposed.to_excel("customers_basic_offers.xlsx")
customer_offer_combined.to_excel("customer_offer_combined.xlsx")
customer_offer_portfolio_combined.to_excel("customer_offer_portfolio_combined.xlsx")

In [1]:
import random

print(random.randint(3, 9))

9


In [13]:
(random.randint(0, 4))

0


In [5]:
import pandas as pd
import numpy as np
import random

df = pd.read_excel("customer_data.xlsx")
df.head()

,gender,age,customer_id,became_member_on,City,Latitude,Longitude,income,offer completed,offer received,...,social_completed,social_received,social_viewed,web_completed,web_received,web_viewed,bogo_completed,bogo_received,discount_completed,discount_received
0,NaN,NaN,3285,20170212,San Francisco,37.76,-122.69,NaN,2,5,...,1,1,2,0,0,1,0,4,2,1
1,F,55.0,9804,20170715,San Francisco,37.76,-122.69,112000.0,1,2,...,0,0,0,0,0,0,1,0,0,2
2,NaN,NaN,4826,20180712,Austin,30.31,-97.76,NaN,0,2,...,0,0,1,0,0,0,0,2,0,0
3,F,75.0,6225,20170509,Boston,42.33,-71.02,100000.0,3,4,...,0,0,0,0,0,0,0,3,3,1
4,NaN,NaN,12377,20170804,Seattle,47.62,-122.35,NaN,0,5,...,0,0,0,0,0,0,0,3,0,2


In [2]:
sample = df
max_value = 9
#keywords are completed, received, viewed
for indx in sample.index:
    keywords = ["received", "viewed", "completed"]

    for keyword in keywords:
        # sample["offer completed"]
        num_offer_completed = int(sample["offer "+keyword][indx])
#         if "received" in keyword:
#             num_offer_completed = max_value
#             sample["offer "+keyword][indx] = random.randint(1, num_offer_completed)
#             num_offer_completed = sample["offer "+keyword][indx]
#         elif "viewed" in keyword:
#             num_offer_completed = sample["offer received"][indx]
#             sample["offer "+keyword][indx] = random.randint(1, num_offer_completed)
#             num_offer_completed = sample["offer "+keyword][indx]
#         elif "completed" in keyword:
#             num_offer_completed = sample["offer viewed"][indx]
#             sample["offer "+keyword][indx] = random.randint(1, num_offer_completed)
#             num_offer_completed = sample["offer "+keyword][indx]
        
        #email_completed, mobile_completed, social_completed, web_completed == offer completed
        num_email_completed = random.randint(0, num_offer_completed)
        num_mobile_completed = random.randint(0, num_offer_completed-num_email_completed)
        num_social_completed = random.randint(0, num_offer_completed-num_email_completed-num_mobile_completed)
        num_web_completed = num_offer_completed-num_email_completed-num_mobile_completed-num_social_completed

        sample["email_"+keyword][indx] = int(num_email_completed)
        sample["mobile_"+keyword][indx] = int(num_mobile_completed)
        sample["social_"+keyword][indx] = int(num_social_completed)
        sample["web_"+keyword][indx] = int(num_web_completed)

        #bogo_completed, discount_completed == offer completed
        num_bogo_completed = random.randint(0, num_offer_completed)
        num_discount_completed = num_offer_completed-num_bogo_completed

        if "viewed" in keyword:
            continue
        else:
            sample["bogo_"+keyword][indx] = int(num_bogo_completed)
            sample["discount_"+keyword][indx] = int(num_discount_completed)
            
sample.to_excel("modified-customer-data-2.xlsx")

<ipython-input-2-287e38b1de45>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["email_"+keyword][indx] = int(num_email_completed)
<ipython-input-2-287e38b1de45>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["mobile_"+keyword][indx] = int(num_mobile_completed)
<ipython-input-2-287e38b1de45>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["social_"+keyword][indx] = int(num_social_completed)
<ipython-input-2-2

In [12]:
sample = df
#keywords are completed, received, viewed

for indx in sample.index:
    keywords = ["received", "viewed", "completed"]
    
    for keyword in keywords:
        # sample["offer completed"]
        #email_completed, mobile_completed, social_completed, web_completed == offer completed
        if "received" in keyword:
            # offer received = email received + mobile received + social received + web received
            num_offer = int(sample["offer "+keyword][indx])
            num_email_received = random.randint(0, num_offer)
            num_mobile_received = random.randint(0, num_offer-num_email_received)
            num_social_received = random.randint(0, num_offer-num_email_received-num_mobile_received)
            num_web_received = num_offer-num_email_received-num_mobile_received-num_social_received
        
            sample["email_"+keyword][indx] = int(num_email_received)
            sample["mobile_"+keyword][indx] = int(num_mobile_received)
            sample["social_"+keyword][indx] = int(num_social_received)
            sample["web_"+keyword][indx] = int(num_web_received)

            # offer received = bogo received + discount received
            num_bogo_received = random.randint(0, num_offer)
            num_discount_received = num_offer-num_bogo_received
            
            sample["bogo_"+keyword][indx] = int(num_bogo_received)
            sample["discount_"+keyword][indx] = int(num_discount_received)
        
        elif "viewed" in keyword:
            num_offer = int(sample["offer "+keyword][indx])           
            num_email_viewed = random.randint(0, min(num_email_received, num_offer))
            num_mobile_viewed = random.randint(0, min(num_mobile_received, num_offer-num_email_viewed))
            num_social_viewed = random.randint(0, min(num_social_received, num_offer-num_email_viewed-num_mobile_viewed))
            num_web_viewed = random.randint(0, min(num_web_received, num_offer-num_email_viewed-num_mobile_viewed-num_social_viewed))
            
            sample["email_"+keyword][indx] = int(num_email_viewed)
            sample["mobile_"+keyword][indx] = int(num_mobile_viewed)
            sample["social_"+keyword][indx] = int(num_social_viewed)
            sample["web_"+keyword][indx] = int(num_web_viewed)
        
        elif "completed" in keyword:
            num_offer = int(sample["offer "+keyword][indx])
            num_email_completed = random.randint(0, min(num_email_viewed, num_offer))
            num_mobile_completed = random.randint(0, min(num_mobile_viewed, num_offer-num_email_completed))
            num_social_completed = random.randint(0, min(num_social_viewed, num_offer-num_email_completed-num_mobile_completed))
            num_web_completed = random.randint(0, min(num_web_viewed, num_offer-num_email_completed-num_mobile_completed-num_social_completed))
            
            sample["email_"+keyword][indx] = int(num_email_completed)
            sample["mobile_"+keyword][indx] = int(num_mobile_completed)
            sample["social_"+keyword][indx] = int(num_social_completed)
            sample["web_"+keyword][indx] = int(num_web_completed)
            
            # issues idhar haiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
            num_bogo_completed = random.randint(0, min(num_bogo_received, num_offer))
            num_discount_completed = random.randint(0, min(num_discount_received, num_offer-num_bogo_completed))
            
            sample["bogo_"+keyword][indx] = int(num_bogo_completed)
            sample["discount_"+keyword][indx] = int(num_discount_completed)
sample.to_excel("modified-customer-data-3.xlsx")

In [9]:
display(sample[["offer completed", "email_completed", "mobile_completed", "social_completed", "web_completed"]])
display("\n")
display(sample[["offer received", "email_received", "mobile_received", "social_received", "web_received"]])
display("\n")
display(sample[["offer viewed", "email_viewed", "mobile_viewed", "social_viewed", "web_viewed"]])

,offer completed,email_completed,mobile_completed,social_completed,web_completed
0,2,0,0,0,0
1,1,0,0,0,0
2,0,0,0,0,0
3,3,0,0,1,0
4,0,0,0,0,0


'\n'

,offer received,email_received,mobile_received,social_received,web_received
0,5,5,0,0,0
1,2,0,2,0,0
2,2,1,0,0,1
3,4,2,1,1,0
4,5,2,2,0,1


'\n'

,offer viewed,email_viewed,mobile_viewed,social_viewed,web_viewed
0,5,0,0,0,0
1,0,0,0,0,0
2,2,0,0,0,1
3,4,0,1,1,0
4,3,1,0,0,0


In [12]:
display(sample[["offer completed", "bogo_completed", "discount_completed"]])
display("\n")
display(sample[["offer received", "bogo_received", "discount_received"]])

,offer completed,bogo_completed,discount_completed
0,2,1.0,1.0
1,2,1.0,1.0
2,2,1.0,1.0
3,2,2.0,0.0
4,2,2.0,0.0
5,2,0.0,2.0
6,2,2.0,0.0
7,2,1.0,1.0
8,2,1.0,1.0
9,2,1.0,1.0


'\n'

,offer received,bogo_received,discount_received
0,5,1.0,4.0
1,5,0.0,5.0
2,5,1.0,4.0
3,5,4.0,1.0
4,5,2.0,3.0
5,5,3.0,2.0
6,5,5.0,0.0
7,5,0.0,5.0
8,5,0.0,5.0
9,5,2.0,3.0


In [28]:
sample.to_excel("modified-profile-output.xlsx")

In [3]:
pd.set_option('mode.chained_assignment',None)
# This code will not complain!
# pd.reset_option("mode.chained_assignment")

In [ ]:
# Would it be possible for you to create a row for each customer with existing income, 
# gender, age, offer completed, offer received, offer viewed, and randomly generate values for the following fields :

 

# Email_received, email_viewed, email_completed

# Mobile_received, mobile_viewed, mobile_completed

# Social_received, social_viewed, social_completed,

# Web_received, web_viewed, web_completed

 

# Bogo_received, bogo_Completed

# Discount_received, discount_Completed

 

# If generating the above columns, please ensure that the sum of email mobile social web is less than
# or equal to the total received, viewed, and completed,

# and that the sum of bogo and discount is less than or equal to the total received and completed as well.